In [72]:
#import libraries

import pandas as pd
import numpy as np
import re

In [73]:
df = pd.read_csv(r"C:\Users\PC\Downloads\indeed_job_data.csv")

In [74]:
df.head()

,Title,Company,Location,Salary,Job Type,Date Posted,Summary
0,"PO, Research & Data Analyst",The Elevation Church,Nigeria,NaN,Full-time,PostedJust posted,Train volunteers on the use of M&E frameworks ...
1,QA Tester - Intern,mDoc Healthcare,Lagos,NaN,NaN,EmployerActive 6 days ago,Advanced knowledge of data security and encryp...
2,Data Analyst - Healthcare,eMedicStore,Lagos,NaN,NaN,EmployerActive 5 days ago,Implement new data analysis methodologies and ...
3,HR Analyst,Jobrole Consulting Limited,Ikeja,"₦150,000 - ₦200,000 a month",Full-time,PostedPosted 2 days ago,Analyze and interpret data to generate actiona...
4,Business Intelligence Analyst,TSL Metroline Limited,Lagos,NaN,NaN,PostedPosted 1 day ago,Analyse business information/ data to identify...


In [75]:
# display information about our dataset

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        203 non-null    object
 1   Company      200 non-null    object
 2   Location     203 non-null    object
 3   Salary       15 non-null     object
 4   Job Type     101 non-null    object
 5   Date Posted  203 non-null    object
 6   Summary      203 non-null    object
dtypes: object(7)
memory usage: 11.2+ KB


In [76]:
#identify missing values

missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values)

Missing Values:
 Title            0
Company          3
Location         0
Salary         188
Job Type       102
Date Posted      0
Summary          0
dtype: int64


In [77]:
#Fill missing Company values with a default value

df['Company'].fillna('Unknown', inplace=True)

#Fill missing Job Type values with a default value

df['Job Type'].fillna('Unknown', inplace=True)


In [78]:
#This gives us an idea of the inconsistencies we'll need  for
df["Salary"].unique()

array([nan, '₦150,000 - ₦200,000 a month', '₦100,000 - ₦150,000 a month',
       '₦200,000 a month', '₦70,000 - ₦150,000 a month',
       '₦90,000 - ₦130,000 a month', '₦300,000 - ₦350,000 a month',
       '₦450,000 - ₦600,000 a month', '₦200,000 - ₦250,000 a month',
       '₦220,000 a month', '₦60,000 - ₦80,000 a month'], dtype=object)

In [79]:
#Removing naira symbols, commas, whitespace and non-numeric characters
df["Salary"] = df["Salary"].str.replace('[₦,a-zA-Z\s]', '', regex=True)


In [80]:
# Convert values in range to average
def convert_to_average(salary):
    if isinstance(salary, str) and '-' in salary:
        # Extract lower and upper range values
        lower, upper = map(int, salary.split('-'))
        return (lower + upper) / 2
    else:
        return salary

df['Salary'] = df['Salary'].apply(convert_to_average)

In [81]:
#Convert to numeric
df["Salary"] = df["Salary"].astype(float)

In [82]:
#input missing salary values with the median of Salary
df["Salary"].fillna(df["Salary"].median(), inplace=True)

In [83]:
df["Salary"].unique()

array([175000., 125000., 200000., 110000., 325000., 525000., 225000.,
       220000.,  70000.])

In [84]:
#identify missing values

missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values)

Missing Values:
 Title          0
Company        0
Location       0
Salary         0
Job Type       0
Date Posted    0
Summary        0
dtype: int64


In [85]:
# Job type column has things like 'Full Time' and 'Full time +1' treated the same way.

df["Job Type"].unique()

array(['Full-time', 'Unknown', 'Temporary', 'Full-time +1', 'Contract',
       'Permanent'], dtype=object)

In [86]:
#Replace the inconsistent value in Job Type Column

df["Job Type"] = df["Job Type"].replace({"Full-time +1": "Full-time"})

In [87]:
#Date posted column has a messy format

#Remove Unnecessary Words
df["Date Posted"] = df["Date Posted"].str.replace("Posted", '') #Remove 'Posted' phrase
df["Date Posted"] = df["Date Posted"].str.replace("EmployerActive", '') #Remove 'EmployerActive' phrase
#Standardize 'Today'
df["Date Posted"] = df["Date Posted"].str.replace("Today", "0 days ago") #Replace "Today" with "0 days ago"

In [89]:
#Define custom function to handle "30+ days ago"
def convert_to_date(value):
    if '+' in value:
        return pd.Timestamp.now().normalize() - pd.Timedelta(days=30)
    elif value.isdigit(): #Check if the value is a digit
        return pd.Timestamp.now().normalize() - pd.Timedelta(days=int(value.split()[0]))
    else:
        #Return a default date for non-numeric values
        return pd.Timestamp.now().normalize() - pd.Timedelta(days=1) #Example: 1 day ago

df['Date Posted'] = df['Date Posted'].apply(convert_to_date)

In [92]:
# Function to remove special characters and non-letter characters
def clean_summary(summary):
    # Define regular expression pattern to match non-letter characters
    pattern = r'[^a-zA-Z\s]'
    # Replace non-letter characters with a space
    cleaned_summary = re.sub(pattern, ' ', summary)
    # Remove extra whitespaces
    cleaned_summary = ' '.join(cleaned_summary.split())
    return cleaned_summary

# Apply the clean_summary function to the Summary column
df['Summary'] = df['Summary'].apply(clean_summary)

In [93]:
df.head()

,Title,Company,Location,Salary,Job Type,Date Posted,Summary
0,"PO, Research & Data Analyst",The Elevation Church,Nigeria,175000.0,Full-time,2024-05-13,Train volunteers on the use of M E frameworks ...
1,QA Tester - Intern,mDoc Healthcare,Lagos,175000.0,Unknown,2024-05-13,Advanced knowledge of data security and encryp...
2,Data Analyst - Healthcare,eMedicStore,Lagos,175000.0,Unknown,2024-05-13,Implement new data analysis methodologies and ...
3,HR Analyst,Jobrole Consulting Limited,Ikeja,175000.0,Full-time,2024-05-13,Analyze and interpret data to generate actiona...
4,Business Intelligence Analyst,TSL Metroline Limited,Lagos,175000.0,Unknown,2024-05-13,Analyse business information data to identify ...


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Title        203 non-null    object        
 1   Company      203 non-null    object        
 2   Location     203 non-null    object        
 3   Salary       203 non-null    float64       
 4   Job Type     203 non-null    object        
 5   Date Posted  203 non-null    datetime64[ns]
 6   Summary      203 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 11.2+ KB


In [95]:
#Save the cleaned Dataframe to a CSV File

df.to_csv("cleaned_job_data.csv", index=False)